# --- Day 4: Passport Processing ---

## Imports

In [1]:
import tensorflow as tf

In [ ]:
byr (Birth Year)
cid (Country ID)
ecl (Eye Color)
eyr (Expiration Year)
hcl (Hair Color)
hgt (Height)
iyr (Issue Year)
pid (Passport ID)

## Part 1

In [27]:
def check_validity_of_kv_pair(kv_pair):
    key, val = kv_pair[0], kv_pair[1]
    
    if key == 'byr':
        if tf.strings.regex_full_match(val, '^[0-9][0-9][0-9][0-9]$'):
            x = tf.strings.to_number(val)
            return x >= 1920 and x <= 2002
        return False
    
    if key == 'iyr':
        if tf.strings.regex_full_match(val, '^[0-9][0-9][0-9][0-9]$'):
            x = tf.strings.to_number(val)
            return x >= 2002 and x <= 2020
        return False
    
    if key == 'eyr':
        if tf.strings.regex_full_match(val, '^[0-9][0-9][0-9][0-9]$'):
            x = tf.strings.to_number(val)
            return x >= 2020 and x <= 2030
        return False
    
    if key == 'hgt' and tf.strings.regex_full_match(val, '^[0-9]+cm$'):
        hgt_s_size = tf.strings.length(val)
        x = tf.strings.to_number(tf.strings.substr(val, 0, hgt_s_size-2))
        return x >= 150 and x <= 193

    if key == 'hgt' and tf.strings.regex_full_match(val, '^[0-9]+in$'):
        hgt_s_size = tf.strings.length(val)
        x = tf.strings.to_number(tf.strings.substr(val, 0, hgt_s_size-2))
        return x >= 59 and x <= 76

    if key == 'hcl':
        return tf.strings.regex_full_match(val, '^#[0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f][0-9a-f]$')
    
    if key == 'ecl':
        return tf.strings.regex_full_match(val, '^(amb|blu|brn|gry|grn|hzl|oth)$')

    if key == 'pid':
        return tf.strings.regex_full_match(val, '^[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]$')

    return False

def check_validity(x): 
    x = x.to_tensor()
    x_decoded = tf.strings.unicode_decode(x[:,0], 'UTF-8').to_tensor()-97
    indices = tf.argsort((26*x_decoded[:,0]) + x_decoded[:,1])
    x = tf.gather(x, indices)
    x = tf.map_fn(
        check_validity_of_kv_pair,
        x,
        fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.bool, name=None)
    )
    
    return tf.math.count_nonzero(x) == 7

@tf.function()
def solve(fname):
    s = tf.io.read_file(fname)
    s = tf.strings.split(s, "\n\n")
    s = tf.map_fn(lambda s: tf.strings.regex_replace(s, '\n', ' '), s)
    s = tf.strings.split(s, ' ')
    s = tf.strings.split(s, ':')
    
    bools = tf.map_fn(check_validity, s, fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.bool, name=None))
    
    return tf.math.count_nonzero(bools)
        

solve('day4_test.txt')

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [29]:
%%time
solve('day4.txt')

CPU times: user 3.07 s, sys: 79.7 ms, total: 3.15 s
Wall time: 2.83 s


<tf.Tensor: shape=(), dtype=int64, numpy=179>

In [321]:
import datetime
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve('day4.txt')
tf.profiler.experimental.stop()